In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from ydata_profiling import ProfileReport

In [4]:
# read the four CSV files into separate DataFrames
dh_causal_lookup = pd.read_csv('dh_causal_lookup.csv')
dh_product_lookup = pd.read_csv('dh_product_lookup.csv')
dh_store_lookup = pd.read_csv('dh_store_lookup.csv')
dh_transactions = pd.read_csv('dh_transactions.csv')

In [5]:
transactions_merged = pd.merge(dh_transactions, dh_product_lookup, on='upc') # Have the same 5197681 row


In [6]:
#drop column that dollar_sales <=0 as it may be a refund = no income
to_drop = transactions_merged[transactions_merged['dollar_sales'] <= 0]
cleaned_transactions = transactions_merged.drop(to_drop.index)
cleaned_transactions.tail()

,upc,dollar_sales,units,time_of_transaction,geography,week,household,store,basket,day,coupon,product_description,commodity,brand,product_size
5197676,2409411034,2.89,1,1634,1,104,488787,166,3286897,722,0,DECECCO TRI COLOR FUSILLI,pasta,Dececco,16 OZ
5197677,4112907705,3.65,1,1629,2,104,503541,308,3298097,725,0,CLASSICO VODKA SAUCE,pasta sauce,Classico,26 OZ
5197678,8692481596,6.93,7,1854,2,104,464514,232,3302319,727,0,EDD OG VEG ALPHABETS,pasta,Edd Og,12 OZ
5197679,4127857643,5.79,1,1528,1,104,342771,193,3308192,728,0,B F PASTA SCE MARINARA,pasta sauce,B F,24 OZ
5197680,2066200269,2.49,1,1834,1,104,228251,152,3312682,724,0,NEWMANS OWN SAUS/PEPP SCE,pasta sauce,Newman's,26 OZ


In [8]:
#Prepare Data for creating timestamp (Assume startdate = 2018-01-01)
date_transactions = cleaned_transactions.loc[:, ['week', 'day', 'time_of_transaction']]
date_transactions

,week,day,time_of_transaction
0,1,1,1100
1,1,1,1222
2,1,1,1826
3,1,1,1807
4,1,1,1927
...,...,...,...
5197676,104,722,1634
5197677,104,725,1629
5197678,104,727,1854
5197679,104,728,1528


In [10]:
# Create new DataFrame just in case
df = date_transactions

# Combine the 'week', 'day', and 'time_of_transaction' columns into a single column called 'timestamp' also, assume that start date is 2018-01-01
df['timestamp'] = pd.to_datetime('2018-01-01') + pd.to_timedelta((df['week'] - 1) * 7 + df['day'] - 1, unit='d') + pd.to_timedelta(df['time_of_transaction'] // 100, unit='h') + pd.to_timedelta(df['time_of_transaction'] % 100, unit='m')

# Drop the original 'week', 'day', and 'time_of_transaction' columns
df = df.drop(['week', 'day', 'time_of_transaction'], axis=1)

# Format the timestamp column to 'YYYY-MM-DD hh:mm' format
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M')

# Print the updated DataFrame to confirm the changes
print(df)

                timestamp
0        2018-01-01 11:00
1        2018-01-01 12:22
2        2018-01-01 18:26
3        2018-01-01 18:07
4        2018-01-01 19:27
...                   ...
5197676  2021-12-13 16:34
5197677  2021-12-16 16:29
5197678  2021-12-18 18:54
5197679  2021-12-19 15:28
5197680  2021-12-15 18:34

[5188096 rows x 1 columns]


In [11]:
#Concat the df
transactions_with_time = pd.concat([cleaned_transactions, df], axis=1)
transactions_with_time = transactions_with_time.drop(['week', 'day', 'time_of_transaction'], axis=1)
transactions_with_time.to_csv('transactions_with_time.csv', index=False)